<a href="https://colab.research.google.com/github/josemoti1999/sysdl_project/blob/master/snn_uci_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Iris dataset

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import copy
import torch.nn.functional as F
import numpy as np

In [0]:
iris = pd.read_csv('iris.csv')
print(iris.shape)
iris.sample(10)

(150, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
25,5.0,3.0,1.6,0.2,Iris-setosa
72,6.3,2.5,4.9,1.5,Iris-versicolor
128,6.4,2.8,5.6,2.1,Iris-virginica
57,4.9,2.4,3.3,1.0,Iris-versicolor
89,5.5,2.5,4.0,1.3,Iris-versicolor
87,6.3,2.3,4.4,1.3,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
130,7.4,2.8,6.1,1.9,Iris-virginica
11,4.8,3.4,1.6,0.2,Iris-setosa
26,5.0,3.4,1.6,0.4,Iris-setosa


In [0]:
mappings = {
   'Iris-setosa': 0,
   'Iris-versicolor': 1,
   'Iris-virginica': 2
}
iris['species'] = iris['species'].apply(lambda x: mappings[x])

In [0]:
iris.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
54,6.5,2.8,4.6,1.5,1
83,6.0,2.7,5.1,1.6,1
49,5.0,3.3,1.4,0.2,0
22,4.6,3.6,1.0,0.2,0
63,6.1,2.9,4.7,1.4,1


In [0]:
iris_train = iris.drop('species', axis=1)

Temporal encoding

In [0]:
def temp_encoding(min_, max_, f, a=0, b=900):
    #900 time steps
    r=max_-min_
    f_new=(((b-a)/r)*f) + (((a*max_)-(b*min_))/r)
    return f_new

In [0]:
for column in iris_train.columns:
    min_=iris[column].min()
    max_=iris[column].max()
    iris[column]=iris[column].apply(lambda x: round(temp_encoding(min_, max_, x), 2))

In [0]:
iris.sample(5)

,sepal_length,sepal_width,petal_length,petal_width,species
126,475.0,300.0,579.66,637.5,2
99,350.0,300.0,472.88,450.0,1
86,600.0,412.5,564.41,525.0,1
26,175.0,525.0,91.53,112.5,0
49,175.0,487.5,61.02,37.5,0


In [0]:
X = iris.drop('species', axis=1).values
y = iris['species'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
class SNN():
    def __init__(self):
        self.synaptic_weights_1 = np.array([[626.94], [829.47], [57.27], [0]])
        self.synaptic_delays_1 = np.array([[8.32], [7.16], [5.57], [0]])
        self.synaptic_weights_2 = np.array([[91]])
        self.synaptic_delays_2 = np.array([[0.63]])
        self.prev_inner_1 = np.zeros([self.synaptic_weights_1.shape[1]])
        self.prev_inner_1 = np.zeros([self.synaptic_weights_2.shape[1]])


    def forward(self, x):
        for time in range(2000):
            x = (x==time/100).float()
            print(x)
            x=np.matmul(x, )

In [0]:
prev_inner_1 = np.zeros(synaptic_weights_1.shape)

In [0]:
prev_inner_1

array([[0.],
       [0.],
       [0.],
       [0.]])

In [0]:
class GeneticAlgorithm():
    def __init__(self, model, target, data):
        self.model=model
        self.target=target
        self.data=data

    def mutate(self, population_size=20, mutation_power=0.02):
        models_list=[]
        models_list.append(self.model)
        for _ in range(population_size):
            child=copy.deepcopy(self.model)
            for value in child.parameters():
                tensor_shape=value.shape
                noise=torch.randn(tensor_shape)
                value+=mutation_power*noise
            models_list.append(child)
        return models_list

    def find_best_model(self, population_size=20, mutation_power=0.02):
        models_list=self.mutate(population_size, mutation_power)
        loss_model_dict={}
        for model in models_list:
            model=model
            output=model(self.data)
            criterion = nn.CrossEntropyLoss()
            loss=criterion(output, self.target)

            loss_model_dict[model]=loss
            for (model,loss) in sorted(loss_model_dict.items(), key=lambda x: x[1], reverse=False):
                model_return=model
                loss_return=loss
                break
        return model_return, loss_return

In [0]:
POPULATION_SIZE=100
MUTATION_POWER=0.1

In [0]:
spiking_model = SpikingNet(device, n_time_steps=900, begin_eval=0)
criterion = nn.CrossEntropyLoss()

In [0]:
epochs = 200
loss_arr = []
with torch.no_grad():
    for i in range(epochs):
        ga=GeneticAlgorithm(spiking_model, y_train, X_train)
        model, loss = ga.find_best_model(POPULATION_SIZE, MUTATION_POWER)
        loss_arr.append(loss)

        print(f'Generation: {i} Loss: {loss}')

Generation: 0 Loss: 37.70244216918945
Generation: 1 Loss: 41.17254638671875
Generation: 2 Loss: 15.856054306030273
Generation: 3 Loss: 52.689998626708984


KeyboardInterrupt: ignored

In [0]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [0]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]
df.sample()

,Y,YHat,Correct
12,0,0,1


In [0]:
df['Correct'].sum() / len(df)

0.9666666666666667